#### Importando pacotes

In [1]:
from math import log, exp
import numpy as np
import random
import matplotlib.pyplot as plt

#### Bases de dados

In [2]:
yanez1 = [3456, 1584, 236, 516, 1820, 452, 432, 1264, 3072, 384, 2448, 32, 360, 998, 656, 180, 244, 1528, 44, 3064, 324, 1528, 348, 336]
yanez2 = [860, 1608, 1134, 2703, 645, 95, 1278, 605, 344, 1054, 680, 405, 367, 2758, 355, 1084, 855, 280, 490, 945, 105, 127, 61, 326, 300]
yanez2_censurados = [300]
bus510 = [778,1821,361,1158,43729,1238,16719,39973,1274,95,531,28382,5938,4412,798,1112,3771,1866,13360,5644,4146,3351,2412,1067,1034,9670,2087,731,9971,961,2633,2033,6275,1433,6599,7361,26298,2971,2151,3518,984,4554,288,8878,13908,22483,18255,84,7576,2478,503,3242,664,1217,6938,300]
wang2 = [27.51, 340.01, 27, 1.12, 11.11, 25.74, 81.68, 341.4, 9.28, 88.17, 86.34, 318.44, 323.12, 169.63, 76.43,471.23, 32.4, 86.43, 83.18, 196.27, 70.91, 432.42, 87.75, 81.01, 220.05, 91.7, 82.17, 92.98]
machine3sc = [16553, 638, 27321, 12306, 16794, 361, 7053, 32811,46043, 74982, 2489, 23154, 26305, 8086, 1427,53756, 108, 16, 19873, 17728, 69437, 19673, 13813,18029, 2899, 33965, 3095, 22967, 51293, 103155,65887, 9089, 273, 3160, 7379, 1932, 24091, 25953,40392, 657, 7313, 7681, 4896, 878, 27831, 1308,37603, 830, 4305, 26901, 506, 1155, 6835, 13077,2219, 12223, 74, 15766, 19464, 2564, 3377, 6743,49, 4460, 624, 7944, 1855, 37265, 13417, 18926,53737, 7853, 4747, 9266, 7482, 21793, 848]

#### Equações 1

In [3]:
# 0 < b < 10 & 0 < q < 1
#nas funções: a, b, q são os parâmetros. tt = 0 significa que é terminada em falha, e tt = 1 significa que é terminada em tempo.

def difa(dados, b, q, tt = 0): #Equação log-verosimilhança derivada em relação a "q" com alfa isolado
    T = dados[-1]
    t1 = dados[0]
    n = len(dados)
    a = ((sum(list(map(lambda i: (dados[i] + q*sum(dados[:i-1]))**b -(q*sum(dados[:i-1]))**b, np.arange(1, len(dados)).tolist()))) + t1**b + tt * ((T + q*sum(dados))**b + (q*sum(dados))**b))/n)**(1/b)
    return a

def difb(dados, a, b, q, tt = 0): #Equação log-verosimilhança derivada em relação a "b" para ser igualada a 0
    T = dados[-1]
    t1 = dados[0]
    n = len(dados)
    beta = (n/b + log(t1) - n*log(a) - ((t1/a)**b)*log(t1/a)) + sum(list(map(lambda i: log(dados[i] + q*sum(dados[:i])) - (((dados[i] + q*sum(dados[:i]))/a)**b) * log((dados[i] + q*sum(dados[:i]))/a) + (((q*sum(dados[:i]))/a)**b)*log(q*sum(dados[:i])/a), np.arange(1,len(dados)).tolist()))) + tt*( 0 - (((T + q*sum(dados))/a)**b) * log((T + q*sum(dados))/a) + ((q*sum(dados)/a)**b) * log(q*sum(dados)/a))
    return beta

def difq(dados, a, b, q, tt=0): #Equação log-verosimilhança derivada em relação a "q" para ser igualada a 0
    T = dados[-1]
    t1 = dados[0]
    n = len(dados)
    que = (b-1)*sum(list(map(lambda i: (sum(dados[:i]))/(dados[i] + q*sum(dados[:i])) , np.arange(1,len(dados)).tolist()))) + (b*q**(b-1)/(a**b))*sum(list(map(lambda i: sum(dados[:i])**b, np.arange(1,len(dados)).tolist()))) - (b/(a**b))*sum(list(map(lambda i: ((dados[i] + q*sum(dados[:i]))**(b-1)) * sum(dados[:i]), np.arange(1,len(dados)).tolist()))) + tt*( ((b/q)*(q*sum(dados)/a)**b) - b*(((T + q*sum(dados))/a)**b) * (sum(dados)/(T + q*sum(dados))))
    return que

def CDF_PRG(dados, a, b, q):
    prob = []
    for i in range(0,len(dados)):
        F = 1 - exp(((q*sum(dados[:-1]))/a)**b - ((dados[i] + q*sum(dados[:-1]))/a)**b)
        prob.append(F)
    return prob

In [4]:
def montecarlo(dados, N, tt = 0, tolerancia = 0.1): #Simulação de montecarlo, onde N é o número de simulações desejadas
    tol = tolerancia # Tolerância para qual o programa irá parar e retornar os valores de alfa, beta e q para a iteração atual
    for i in range(0, N): # Loop da simulação
        b = random.uniform(0.001, 10) # Gerador de números aleatórios para beta entre 0 e 10 (distribuido uniformemente)
        q = random.uniform(0.001, 1) # Gerador de números aleatórios para beta entre 0 e 1 (distribuído uniformemente)
        a = difa(dados, b, q)
        if a > 0:       
            result_b = difb(dados, a, b, q, tt)
            result_q = difq(dados, a, b, q, tt)
            if 0 < result_b < tol and 0 < result_q < tol: # Caso a iteração de b e q deem menor que 0.1 como resultado para as equações derivadas, retorna os parâmetros
                return f"alfa = {a}, beta = {b}, q = {q}"
        else:
            continue
    return "Deu ruim!" # Caso a simulação não encontre valores menores que a tolerância para o número de iterações desejado

montecarlo(wang2, 100000, tolerancia=0.3)

'alfa = 104.07923978330344, beta = 0.8218662763220422, q = 0.0654008937757586'

In [5]:
# Simulação Monte Carlo para tempos de falha

def simulacao(a, b, q, T):
    failures = []
    u_list = []
    if len(failures) == 0: #gerando o primeiro valor
        u = random.uniform(0,1)
        x = a * ( - log(1-u) )**(1/b)
        failures.append(x)
        u_list.append(u)
        if sum(failures) > T: #
            return [failures, u_list]
    while sum(failures) < T:
        u = random.uniform(0,1)
        x = a*((sum(failures[:-1])/a)**b - log(1-u))**(1/b) - sum(failures[:-1]) #equaçao do slide da aula 6 para achar x
        failures.append(x)
        u_list.append(u)
    return [failures, u_list]

z = sum(wang2)
sim = simulacao(111.76281138389334, 0.9320177386067728, 1, z) # aqui chama a função

x = sim[0]
y = sim[1]
p = CDF_PRG(sim[0], 111.76281138389334, 0.9320177386067728, 1)

# n = sum(machine3sc)/(np.mean(x))
# print(n)

# print(len(x))
# print(len(y))

# plt.scatter(x,y, color='orange')
# plt.xlabel('Tempo entre falhas')
# plt.ylabel('Probabilidade')
# plt.title('PDF com falhas')
# plt.show

def erro_padrao(o, m):
    erro = 0
    for i in range(0, len(o)):
        erro += abs(o[i] - m[i])
    return erro/len(o)

erro = erro_padrao(x,p)
print(erro)

111.19272804659477


In [6]:
def erro_padrao(o, m):
    erro = 0
    for i in range(0, len(o)):
        erro += abs(o[i] - m[i])
    return erro/len(o)

erro = erro_padrao(x,p)
print(erro)

111.19272804659477


In [200]:
def montecarlo(dados, N, tt = 0, tolerancia = 0.1): #Simulação de montecarlo, onde N é o número de simulações desejadas
    tol = tolerancia # Tolerância para qual o programa irá parar e retornar os valores de alfa, beta e q para a iteração atual
    for i in range(0, N): # Loop da simulação
        b = random.uniform(0.001, 10) # Gerador de números aleatórios para beta entre 0 e 10 (distribuido uniformemente)
        q = 1 # Gerador de números aleatórios para beta entre 0 e 1 (distribuído uniformemente)
        a = difa(dados, b, q)
        if a > 0:       
            result_b = difb(dados, a, b, q, tt)
            result_q = difq(dados, a, b, q, tt)
            if 0 < result_b < tol: # Caso a iteração de b e q deem menor que 0.1 como resultado para as equações derivadas, retorna os parâmetros
                return f"alfa = {a}, beta = {b}, q = {q}"
        else:
            continue
    return "Deu ruim!" # Caso a simulação não encontre valores menores que a tolerância para o número de iterações desejado

montecarlo(wang2, 100000, tolerancia=0.1)


bus510 = 5335
machine3sc = 6439
wang2 = 221

'alfa = 111.76281138389334, beta = 0.9320177386067728, q = 1'